In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
from pathlib import Path

In [2]:
db_path = Path("test.db")

if db_path.exists():
    db_path.unlink()

In [3]:
engine = create_engine("sqlite:///test.db")

In [4]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE IF NOT EXISTS test (a INT, b TEXT)"))

In [5]:
with engine.connect() as conn:
    conn.execute(text("INSERT INTO test (a, b) VALUES (1, 'foo')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (2, 'bar')"))
    conn.execute(text("INSERT INTO test (a, b) VALUES (3, 'baz')"))
    conn.commit()

In [ ]:
df = pd.read_sql_query("SELECT * FROM test", engine)

df

,a,b
0,1,foo
1,2,bar
2,3,baz


In [7]:
# add new column to dataframe

df["c"] = [4, 5, 6]
df["d"] = ["qux", "quux", "quuz"]

df

,a,b,c,d
0,1,foo,4,qux
1,2,bar,5,quux
2,3,baz,6,quuz


In [8]:
from sqlmigrately import migrate_table

In [9]:
migrate_table("test", df, engine, push_data=True)

2024-11-23 19:09:48.811 | INFO     | sqlmigrately.main:migrate_table:21 - Detected new columns: [{'name': 'c', 'type': dtype('int64')}, {'name': 'd', 'type': dtype('O')}]
2024-11-23 19:09:48.812 | INFO     | sqlmigrately.utils:alter_table:83 - Executing: ALTER TABLE test ADD COLUMN c int64
2024-11-23 19:09:48.835 | INFO     | sqlmigrately.utils:alter_table:83 - Executing: ALTER TABLE test ADD COLUMN d object
2024-11-23 19:09:48.856 | INFO     | sqlmigrately.main:migrate_table:31 - Appending data to test


In [10]:
new_table = pd.read_sql_query("SELECT * FROM test", engine)

new_table

,a,b,c,d
0,1,foo,NaN,None
1,2,bar,NaN,None
2,3,baz,NaN,None
3,1,foo,4.0,qux
4,2,bar,5.0,quux
5,3,baz,6.0,quuz


In [11]:
# delete column from dataframe

del new_table["a"]

new_table

,b,c,d
0,foo,NaN,None
1,bar,NaN,None
2,baz,NaN,None
3,foo,4.0,qux
4,bar,5.0,quux
5,baz,6.0,quuz


In [12]:
migrate_table("test", new_table, engine, push_data=True, remove_cols=True)

2024-11-23 19:09:48.936 | INFO     | sqlmigrately.main:migrate_table:26 - Detected removed columns: [{'name': 'a', 'type': INTEGER()}]
2024-11-23 19:09:48.938 | INFO     | sqlmigrately.utils:alter_table:83 - Executing: ALTER TABLE test DROP COLUMN a
2024-11-23 19:09:48.960 | INFO     | sqlmigrately.main:migrate_table:31 - Appending data to test


In [13]:
final_table = pd.read_sql_query("SELECT * FROM test", engine)

final_table

,b,c,d
0,foo,NaN,None
1,bar,NaN,None
2,baz,NaN,None
3,foo,4.0,qux
4,bar,5.0,quux
5,baz,6.0,quuz
6,foo,NaN,None
7,bar,NaN,None
8,baz,NaN,None
9,foo,4.0,qux


In [ ]:
# delete database
db_path.unlink()